In [1]:
import pandas as pd
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import numpy as np
nltk.download("punkt", quiet=True)

True

In [2]:
file_path = r'C:\Users\ungdu\Downloads\Test_Chatgpt\test_data.csv' #import data
data = pd.read_csv(file_path)
data

,Câu hỏi,Câu trả lời
0,Các quả có mùi vị như thế nào?,Quả cam ngon. Quả táo dở. Quả chanh chua. Quả ...
1,Các quả có hình dáng như thế nào?,"Quả cam có hình tròn. Quả táo có hình tròn, hơ..."


In [ ]:
import pandas as pd
import nltk
from IPython.display import display

# Tải bộ dữ liệu phân tách câu của NLTK nếu chưa có
nltk.download('punkt')


# Danh sách chứa tất cả các câu đã phân tách
all_sentences = []

# Lặp qua từng dòng của DataFrame
for _, row in data.iterrows():
    # Lấy nội dung cột "Câu trả lời"
    text = row.get("Câu hỏi", "")
    
    # Kiểm tra nếu text không rỗng và là chuỗi
    if text and isinstance(text, str):
        # Chia văn bản thành các câu
        sentences = nltk.sent_tokenize(text)
        
        # Hiển thị từng câu đã tách (chỉ để kiểm tra)
        display(sentences)
        
        # Thêm các câu này vào danh sách tổng hợp
        all_sentences.extend(sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ungdu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Các quả có mùi vị như thế nào?']

['Các quả có hình dáng như thế nào?']

In [ ]:
# Tạo DataFrame từ danh sách các câu
sentences_df = pd.DataFrame({'Câu hỏi': all_sentences})

# Lưu DataFrame vào file CSV
output_filename = 'extracted_sentences.csv'
sentences_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"Đã lưu {len(all_sentences)} câu vào file '{output_filename}'.")

Đã lưu 2 câu vào file 'extracted_sentences.csv'.


In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Đọc file CSV chứa các câu đã được tách
sentences_df = pd.read_csv('extracted_sentences.csv', encoding='utf-8-sig')
print(sentences_df)
# Hiển thị số lượng câu đã được tải
print(f"Đã tải {len(sentences_df)} câu từ 'extracted_sentences.csv'.")

                             Câu hỏi
0     Các quả có mùi vị như thế nào?
1  Các quả có hình dáng như thế nào?
Đã tải 2 câu từ 'extracted_sentences.csv'.


In [6]:
# Bước 3: Vector hóa các câu sử dụng TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences_df['Câu hỏi'])

# Hiển thị kích thước ma trận TF-IDF
print(f"Ma trận TF-IDF có kích thước: {tfidf_matrix.shape}")

# Bước 2: Tính độ tương đồng giữa các câu bằng cosine similarity
# Tính ma trận độ tương đồng giữa các câu
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Hiển thị một phần ma trận độ tương đồng
print("Mẫu độ tương đồng giữa các câu:")
print(cosine_sim[:5, :5])

Ma trận TF-IDF có kích thước: (2, 10)
Mẫu độ tương đồng giữa các câu:
[[1.         0.60297482]
 [0.60297482 1.        ]]


In [7]:
# 1. Lưu Ma Trận TF-IDF
# Chuyển đổi ma trận TF-IDF thành DataFrame
# Lưu ý: Nếu số lượng từ vựng (features) lớn, việc lưu trữ có thể tiêu tốn nhiều dung lượng
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Lưu DataFrame TF-IDF vào CSV
tfidf_output_filename = 'tfidf_matrix.csv'
tfidf_df.to_csv(tfidf_output_filename, index=False, encoding='utf-8-sig')

print(f"Đã lưu ma trận TF-IDF vào file '{tfidf_output_filename}'.")

Đã lưu ma trận TF-IDF vào file 'tfidf_matrix.csv'.


In [8]:
# 2. Lưu Ma Trận Cosine Similarity
# Chuyển đổi ma trận cosine similarity thành DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim, index=sentences_df.index, columns=sentences_df.index)

# Lưu DataFrame cosine similarity vào CSV
cosine_sim_output_filename = 'cosine_similarity_matrix.csv'
cosine_sim_df.to_csv(cosine_sim_output_filename, index=True, encoding='utf-8-sig')

print(f"Đã lưu ma trận cosine similarity vào file '{cosine_sim_output_filename}'.")

Đã lưu ma trận cosine similarity vào file 'cosine_similarity_matrix.csv'.


In [9]:
# Bước 4: Gép các câu lại với nhau dựa trên độ tương đồng

# Thiết lập ngưỡng độ tương đồng
SIMILARITY_THRESHOLD = 0.3 # Bạn có thể điều chỉnh ngưỡng này

# Danh sách để lưu các chunk
chunks = []
current_chunk = sentences_df.iloc[0]['Câu hỏi']  # Bắt đầu với câu đầu tiên trong cột "Câu hỏi"

for i in range(1, len(sentences_df)):
    prev_sentence = sentences_df.iloc[i - 1]['Câu hỏi']
    current_sentence = sentences_df.iloc[i]['Câu hỏi']
    
    # Vector hóa câu hiện tại và câu trước đó
    prev_vector = vectorizer.transform([prev_sentence])
    current_vector = vectorizer.transform([current_sentence])
    
    # Tính độ tương đồng giữa câu trước và câu hiện tại
    similarity = cosine_similarity(prev_vector, current_vector)[0][0]
    
    print(f"Độ tương đồng giữa câu {i} và câu {i+1}: {similarity:.2f}")
    
    if similarity >= SIMILARITY_THRESHOLD:
        # Gép câu hiện tại vào chunk hiện tại
        current_chunk += ' ' + current_sentence
    else:
        # Thêm chunk hiện tại vào danh sách và bắt đầu chunk mới
        chunks.append(current_chunk)
        current_chunk = current_sentence

# Thêm chunk cuối cùng vào danh sách
chunks.append(current_chunk)

# Tạo DataFrame từ các chunk
chunks_df = pd.DataFrame({'Chunk': chunks})

# Lưu các chunk vào file CSV mới
output_chunk_filename = 'chunked_sentences.csv'
chunks_df.to_csv(output_chunk_filename, index=False, encoding='utf-8-sig')

print(f"Đã lưu {len(chunks)} chunks vào file '{output_chunk_filename}'.")


Độ tương đồng giữa câu 1 và câu 2: 0.60
Đã lưu 1 chunks vào file 'chunked_sentences.csv'.


In [10]:
# --- Bước 7: Thêm Cột "Chunk" vào Dữ Liệu Gốc ---

# Khởi tạo cột 'Chunk' trong sentences_df
sentences_df['Chunk'] = None

# Duyệt qua từng chunk và gán cho các câu thuộc chunk đó
for idx, row in chunks_df.iterrows():
    chunk_text = row['Chunk']
    # Sử dụng NLTK để phân tách các câu trong chunk
    chunk_sentences = nltk.sent_tokenize(chunk_text)
    
    for sent in chunk_sentences:
        # Tìm chỉ số của câu trong sentences_df
        match_idx = sentences_df[sentences_df['Câu hỏi'] == sent].index
        if not match_idx.empty:
            sentences_df.at[match_idx[0], 'Chunk'] = chunk_text
        else:
            print(f"Warning: Sentence '{sent}' not found in sentences_df.")

# Kiểm tra các chunk đã được gán
print("\nDữ liệu với cột 'Chunk':")
print(sentences_df)

# Lưu DataFrame sentences_df vào file CSV
output_filename = 'question_with_chunk.csv'
sentences_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\nDữ liệu đã được lưu vào file '{output_filename}'.")



Dữ liệu với cột 'Chunk':
                             Câu hỏi  \
0     Các quả có mùi vị như thế nào?   
1  Các quả có hình dáng như thế nào?   

                                               Chunk  
0  Các quả có mùi vị như thế nào? Các quả có hình...  
1  Các quả có mùi vị như thế nào? Các quả có hình...  

Dữ liệu đã được lưu vào file 'question_with_chunk.csv'.


In [11]:
data

,Câu hỏi,Câu trả lời
0,Các quả có mùi vị như thế nào?,Quả cam ngon. Quả táo dở. Quả chanh chua. Quả ...
1,Các quả có hình dáng như thế nào?,"Quả cam có hình tròn. Quả táo có hình tròn, hơ..."


In [12]:
sentences_df

,Câu hỏi,Chunk
0,Các quả có mùi vị như thế nào?,Các quả có mùi vị như thế nào? Các quả có hình...
1,Các quả có hình dáng như thế nào?,Các quả có mùi vị như thế nào? Các quả có hình...


In [13]:
import pandas as pd

# Tạo danh sách để lưu kết quả
expanded_rows = []

# Duyệt qua từng dòng trong data
for _, data_row in data.iterrows():
    question = data_row['Câu hỏi']  # Câu hỏi từ data
    full_answer = data_row['Câu trả lời']  # Câu trả lời từ data

    # Tìm các chunk tương ứng từ sentences_df
    matching_chunks = sentences_df[sentences_df['Câu hỏi'] == question]
    if not matching_chunks.empty:
        for _, chunk_row in matching_chunks.iterrows():
            chunk = chunk_row['Chunk']  # Lấy giá trị Chunk
            expanded_rows.append({
                'Câu hỏi': question,
                'Câu trả lời': full_answer,
                'Chunk': chunk
            })

# Tạo DataFrame từ danh sách kết quả
test_data_with_chunk = pd.DataFrame(expanded_rows)

# Lưu kết quả vào file CSV
output_file = 'test_data_with_chunk.csv'
test_data_with_chunk.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Dữ liệu đã được ghép và lưu vào file '{output_file}'.")

Dữ liệu đã được ghép và lưu vào file 'test_data_with_chunk.csv'.


In [14]:
data

,Câu hỏi,Câu trả lời
0,Các quả có mùi vị như thế nào?,Quả cam ngon. Quả táo dở. Quả chanh chua. Quả ...
1,Các quả có hình dáng như thế nào?,"Quả cam có hình tròn. Quả táo có hình tròn, hơ..."


In [21]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from chromadb import Client
from chromadb.config import Settings

# Khởi tạo SentenceTransformer
model = SentenceTransformer('keepitreal/vietnamese-sbert')



# Khởi tạo client mới
chroma_client = PersistentClient(Settings(
    persist_directory="./chroma_db"  # Đường dẫn lưu trữ dữ liệu
))


# Tạo collection trong ChromaDB
collection_name = "questions_answers_chunks"
collection = chroma_client.get_or_create_collection(name=collection_name)

# Đọc dữ liệu từ file
data_file = 'test_data_with_chunk.csv'
data = pd.read_csv(data_file, encoding='utf-8-sig')

# Tạo embeddings và lưu dữ liệu
batch_size = 10  # Số lượng câu xử lý mỗi lô
for start_idx in range(0, len(data), batch_size):
    # Lấy batch hiện tại
    batch = data.iloc[start_idx:start_idx + batch_size]
    
    # Tạo embeddings từ cột 'Chunk'
    embeddings = model.encode(batch['Chunk'].tolist())
    
    # Chuẩn bị metadata
    metadata = [
        {
            "Câu hỏi": row['Câu hỏi'],
            "Câu trả lời": row['Câu trả lời'],
            "Chunk": row['Chunk']
        }
        for _, row in batch.iterrows()
    ]
    
    # Tạo danh sách id cho mỗi chunk
    ids = [f"chunk_{start_idx + i}" for i in range(len(batch))]
    
    # Thêm vào collection
    collection.add(
        embeddings=embeddings,
        metadatas=metadata,
        ids=ids
    )

print("Dữ liệu đã được lưu vào ChromaDB.")


NameError: name 'PersistentClient' is not defined

In [22]:
pip install chromadb --upgrade


Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install --use-pep517

Note: you may need to restart the kernel to use updated packages.


ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
pip help install

In [19]:
pip help install


Usage:   
  c:\Users\ungdu\AppData\Local\Programs\Python\Python312\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\ungdu\AppData\Local\Programs\Python\Python312\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\ungdu\AppData\Local\Programs\Python\Python312\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\ungdu\AppData\Local\Programs\Python\Python312\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\ungdu\AppData\Local\Programs\Python\Python312\python.exe -m pip install [options] <archive url/path> ...

Description:
  Install packages from:
  
  - PyPI (and other indexes) using requirement specifiers.
  - VCS project urls.
  - Local project directories.
  - Local or remote source archives.
  
  pip also supports installing from "requirements files", which provide
  an easy way to specify a whole environment to be installed.

Install Opti

In [ ]:
# --- Function for Vector Search ---
def vector_search(prompt, model, collection, top_k=5):
    """
    Perform vector search in ChromaDB collection.
    Args:
        prompt (str): The search query.
        model (SentenceTransformer): The loaded model for embedding.
        collection: The ChromaDB collection object.
        top_k (int): Number of top results to retrieve.
    """
    try:
        # Encode the prompt into an embedding
        prompt_embedding = model.encode([prompt], show_progress_bar=False)[0]
        
        # Perform the search in the collection
        search_results = collection.query(
            query_embeddings=[prompt_embedding],
            n_results=top_k,
            include=["metadatas", "distances"]
        )
        
        # Display the search results
        if search_results and 'metadatas' in search_results:
            print(f"\nSearch Results for: '{prompt}'")
            for i, metadata in enumerate(search_results['metadatas'][0]):
                distance = search_results['distances'][0][i]
                print(f"\nResult {i + 1}:")
                print(f"Chunk: {metadata.get('Chunk')}")
                print(f"Metadata: {metadata.get('Metadata')}")
                print(f"Distance: {distance}")
        else:
            print(f"No results found for query: '{prompt}'")
    except Exception as e:
        print(f"Error during vector search: {e}")

# --- Perform Vector Search ---
prompt = "quả nào ngon"
vector_search(prompt, model, collection, top_k=5)



Search Results for: 'quả nào ngon'

Result 1:
Chunk: Các quả có mùi vị như thế nào? Các quả có hình dáng như thế nào?
Metadata: None
Distance: 55.55916865763455

Result 2:
Chunk: Các quả có mùi vị như thế nào? Các quả có hình dáng như thế nào?
Metadata: None
Distance: 55.55916865763455

Result 3:
Chunk: Các quả có mùi vị như thế nào? Các quả có hình dáng như thế nào?
Metadata: None
Distance: 55.55916865763455

Result 4:
Chunk: Các quả có mùi vị như thế nào? Các quả có hình dáng như thế nào?
Metadata: None
Distance: 55.55916865763455

Result 5:
Chunk: Các quả có mùi vị như thế nào? Các quả có hình dáng như thế nào?
Metadata: None
Distance: 55.55916865763455
